In [24]:
##Loading the Data:

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

node_edge_path = "C:/Users/Chafiaa/Downloads/soc-political-retweet.edges"
node_edge_data = pd.read_csv(node_edge_path, sep=',', header=None, names=['node', 'edge', 'weight'])

node_label_path = "C:/Users/Chafiaa/Downloads/soc-political-retweet.node_labels"
node_label_data = pd.read_csv(node_label_path, sep=',', header=None, names=['node', 'category'])


In [25]:
# Merge DataFrames
merged_data = node_edge_data.merge(node_label_data, left_on='node', right_on='node', how='left')

# Create the graph from merged data, assuming 'source' and 'target' are the edge columns
# and 'label' is the column with node labels
G = nx.from_pandas_edgelist(merged_data, 'node', 'edge', 'category')

# Add node labels as attributes
for _, row in merged_data.iterrows():
    G.nodes[row['node']]['category'] = row['category']

In [26]:
merged_data.head(3)

,node,edge,weight,category
0,12464,7349,1286901355,1
1,6820,12569,1287988962,2
2,4336,15967,1286535938,1


In [27]:
print(G)

Graph with 18470 nodes and 48053 edges


In [28]:
##Create NetworkX Graph Object
weight_nodes = merged_data[['node','edge','weight','category']].drop_duplicates().sort_values(by='node')
G = nx.Graph()
G.add_nodes_from(merged_data['weight'], bipartite=0)
G.add_nodes_from(merged_data['category'], bipartite=1)
G.add_weighted_edges_from([tuple(d) for d in merged_data[['weight','category','node']].values])

In [29]:
list(G.nodes(data=True))[:3]
list(G.nodes(data=True))[-2:]

[(1, {'bipartite': 1}), (2, {'bipartite': 1})]

In [30]:
##Set 'Top' and 'Bottom' Nodes:
weight_nodes = {n for n, d in G.nodes(data=True) if d['bipartite']==0}
weight_nodes = set(G) - weight_nodes

In [31]:
print(bi.density(G, weight_nodes))

0.5025123551676832


In [32]:
def set_colors(G):
    colors = []
    for node, data in G.nodes(data=True):
        if data['bipartite'] == 1: 
            colors.append('olivedrab')   
        else:
            colors.append('cornflowerblue') 
    return colors

In [ ]:
plt.rcParams["figure.figsize"] = (15,15) # set plot size
colors = set_colors(G)  # set colors
#weights = [math.log(edata['attr_dict'][200]) for f, t, edata in G0.edges(data=True)] # set weights

nx.draw(G, with_labels=True, node_color=colors, node_size=200, 
        font_size=10, font_weight='bold', edge_color="skyblue", alpha=0.5)